# STA314 Final Project Report - YouTube Spam Comments

### Problem Statement

**Relevance and importance**







### Statistical Analyses

### Result and Conclusion

### Discussion 

### References

### Code